In [ ]:
######### Import necessary libraries. #########
import pandas as pd # version 1.5.2
import numpy as np # version 1.24.1
import matplotlib.pyplot as plt # version 3.6.3
import seaborn as sns # version 0.12.2
from sklearn import linear_model # version 1.2.0
import warnings # python version 3.9.16

In [ ]:
######### Define helper functions. #########
def bin_transactions(trans):
    """Use Doane's formula to bin transactions.

    Args:
        trans: pandas dataframe with (transformed) transactions in a column
            named "Size".
        
    Returns:
        trans_binned: pandas dataframe with binned transactions.
        b_values: numpy array of bin cutoff points.
        bin_map: map from bins to bin indicies.
        
    """
    
    # Doane's formula to find bin width omega.
    # The formula is implemented in analog to 
    # https://numpy.org/doc/stable/reference/generated/numpy.histogram_bin_edges.html
    n = len(trans)   
    mu = np.mean(trans['Size'])
    sigma = np.std(trans['Size'])
    g1 = np.mean(((trans['Size'] -mu)/sigma)**3)
    sigma_g1 = np.sqrt((6*(n-2))/((n+1)*(n+3)))
    
    k = np.round((1 + np.log2(n) + np.log2(1+abs(g1)/sigma_g1)),0)
    omega = (trans['Size'].max() - trans['Size'].min())/(k-1)
    
    # Find N_min and N_max (see article for definition).
    N_min = int(np.floor(trans['Size'].min() / omega))
    N_max = int(np.ceil(trans['Size'].max() / omega))
    # Bin transactions.
    b_values = [omega*i for i in range(N_min, N_max+1)]
    trans['Bin'] = pd.cut(x=trans['Size'], bins=b_values, right=True)
    
    # Check for number of bins.
    if not len(trans['Bin'].unique()) == int(k):
        warnings.warn('Number of bins not as expected!')

    # Aggregate transactions per bin.
    trans_agg = trans[['Bin','Size']].groupby('Bin').count().reset_index()
    trans_agg['Size'] /= trans_agg['Size'].sum()
    trans_agg = trans_agg.rename(columns={'Size':'Fraction of Transactions'})
    
    # Make and add a map between bins and their indicies.
    bins, bin_index = list(trans['Bin'].cat.categories), np.arange(N_min,N_max+1)
    bin_map = {bin_: index_ for bin_, index_ in zip(bins, bin_index)}
    trans_agg['Bin Index'] = trans_agg['Bin'].map(bin_map)     
    
    return trans_agg, b_values, bin_map

def colors_from_values(values):
    """Find palette colors for bar plot depending on the values to plot.
    
    Args:
        values: pandas series with excess transaction fractions (floats).
        
    Returns:
        palette_cols: palette colors for plot.
    
    """
    
    # Normalize values to run between 0 and 1.
    normalized = (values - min(values)) / (max(values) - min(values))    
    
    # Specific palette to use.
    palette = sns.color_palette('dark:salmon_r', len(values))
    
   # Get color values depending on size.
    indices = np.round(normalized * (len(values) - 1)).astype(np.int32)    
    palette_cols = np.array(palette).take(indices, axis=0)
    
    return palette_cols

def estimate_zeta(trans, l, u, plot=False, fig_name=None):
    """Fit high degree polynomial and estimate zeta.

    Args:
        trans: pandas dataframe with (transformed) transactions in a column
            named "Size".
        l: number of bins (negative integer) below threshold to exclude from analysis.
        u: number of bins (positive integer) above threshold to exclude from analysis.
        plot: should plots be printed (boolean)
        fig_name: name (string) to use when saving plots (only relevant if plot is True)
        
    Returns:
        zeta: estimated zeta metric.
        
    """
    
    # Bin transactions (don't need last two returned arguments.)
    trans_bin, b_values, bin_map = bin_transactions(trans)
    
    # Extract mu^1.
    trans_bin['mu^1'] = trans_bin['Bin'].apply(lambda x: x.mid).astype(float)
    
    # Set define inputs to polynomial.
    p, cols = int(np.round(((len(bin_map))/2),0)), []
    for h in range(1,p+1):
        trans_bin[f'mu^{h}'] = trans_bin['mu^1']**h
        cols.append(f'mu^{h}')
        
    # Get bins to fit.
    X_fit = trans_bin[~trans_bin['Bin Index'].isin(np.arange(l,u))][cols]
    Y_fit = trans_bin[~trans_bin['Bin Index'].isin(np.arange(l,u))]['Fraction of Transactions']

    # Fit model.
    res = linear_model.LinearRegression()
    res.fit(X_fit,Y_fit)

    # Get predictions.
    preds = res.predict(trans_bin[cols])

    # Get threshold index.
    thres_index = trans_bin[[0 in interval for interval in trans_bin['Bin'].values]].index
    
    # Calcualte zeta (note the minus in the zeta calculation as we consider excess
    # transactions, i.e., the order of \hat(n_i) and n_i is switched above the threshold).
    trans_bin['Excess Transactions'] = trans_bin['Fraction of Transactions'] - preds
    below_thres = trans_bin[trans_bin['Bin Index'].isin(np.arange(l,0))]
    over_thres = trans_bin[trans_bin['Bin Index'].isin(np.arange(0,u))]
    
    zeta = below_thres['Excess Transactions'].sum() - over_thres['Excess Transactions'].sum()
    
    # Make a plots.
    if plot == True:
        
        # Get color scheme and make fit figure.
        color_tab = sns.color_palette('tab10')
        fig, ax = plt.subplots(figsize = (12, 6))
        plot = sns.barplot(data=trans_bin, x='Bin Index', y='Fraction of Transactions',
                           color=color_tab[0])
                           
        # Plot predictions and threshold line.
        plot.scatter(trans_bin.index,preds,color=sns.color_palette('tab10')[1], linewidths=3)
        plot.axvline(thres_index[0]+0.5, ls='--', color=sns.color_palette('tab10')[4], linewidth=4)
        
        # Give non-fitted bins red and square markers.
        non_fitted = trans_bin[trans_bin['Bin Index'].isin(np.arange(l,u))].index
        plot.scatter(non_fitted,preds[non_fitted], color=sns.color_palette('tab10')[3], marker='s', linewidths=3)
        
        # Set ylabels and format ticks.                                   
        ax.yaxis.set_major_formatter(lambda x, pos: f'{x*100:.2f}')
        ax.set_ylabel('Fraction of Transactions (%)', labelpad=10, fontsize=26)
        ax.set_xlabel('Bin Index', labelpad=10, fontsize=26)
        plt.xticks(fontsize=20)
        plt.yticks(fontsize=20)
        
        # Save and show plot.
        fig.tight_layout()
        if fig_name:
            plt.savefig(f'{fig_name}_fit.pdf', bbox_inches='tight')
        plt.show()
        
        # Make deviation figure.
        fig, ax = plt.subplots(figsize = (12, 6))        
        deviation_colors = colors_from_values(trans_bin['Excess Transactions'])
        plot = sns.barplot(data=trans_bin, x="Bin Index", y='Excess Transactions', palette=deviation_colors);
        ax.set_ylabel('Excess Transactions (%)', labelpad=10, fontsize=26)
        ax.set_xlabel('Bin Index', labelpad=10, fontsize=26)
        plot.axvline(thres_index[0]+0.5, ls='--', color=sns.color_palette('tab10')[4], linewidth=4)
        plot.scatter([],[]) # Needed to agline observations with the fit plot.
        ax.yaxis.set_major_formatter(lambda x, pos: f'{x*100:.2f}')
        plt.xticks(fontsize=20)
        plt.yticks(fontsize=20)
        fig.tight_layout()
        
        # Save and show plot.
        if fig_name:
            plt.savefig(f'{fig_name}_deviation.pdf', bbox_inches='tight')
        plt.show()       
        
    return zeta

def bootstrap(trans, l, u, iterations=10000):
    """Bootstrap confidence limist for zeta.
    
    Args:
        trans: pandas dataframe with (transformed) transactions in a column
            named "Size".
        l: number of bins (negative int) below threshold to exclude from analysis.
        u: number of bins (positive int) above threshold to exclude from analysis.
        iterations: number of iterations (int) to use for boostrapping.
        
    Returns:
        lower_cl: lower confidence limit.    
    """
    
    zetas = []
    for j in range(0,iterations):     
        trans_bootstrap = trans.sample(frac=1, replace=True, random_state=j)
        zeta = estimate_zeta(trans_bootstrap, l=l, u=u)
        zetas.append(zeta)
    lower_cl = np.percentile(zetas,(5),method='lower')
    
    return lower_cl 

In [ ]:
######### Set simulation variables. #########
params = [{'Type':'A_fake','mu':-2.5,'sigma':1.8,'size':50000},
          {'Type':'B_fake','mu':-2.1,'sigma':2.1,'size':250000},]
rs = [0.000,0.001,0.005]


######### Run simulation. #########
for param in params:
    for r in rs:
        
        ######### Sample data. #########
        print(f"Doing type {param['Type']} with {r*100:.2f}% money laundering.")
        np.random.seed(42)
        sample = np.random.normal(loc=param['mu'], scale=param['sigma'], size=param['size'])
        transactions = pd.DataFrame(data= {'Size': sample})
        # Discard outliers.
        transactions = transactions[(transactions['Size'] < transactions['Size'].quantile(0.999))
                                    &(transactions['Size'] > transactions['Size'].quantile(0.001))]
    
        ######### Add money laundering behavior to the data. #########
        transactions_ml = transactions.copy()
        n = len(transactions_ml)
        
        # Get bins and bin_map for the transactions 
        # (don't need first return with aggregated transactions).
        _, b_values, bin_map = bin_transactions(transactions_ml)
        
        # Bin transactions before introducing money laundering behavior.
        transactions_ml['Bin'] = pd.cut(x=transactions_ml['Size'], bins=b_values, right=True)
        transactions_ml['Bin Index'] = transactions_ml['Bin'].map(bin_map)    
        
        # Is the transaction in a bin where money laundering occures?
        transactions_ml['ML Candicate'] = transactions_ml['Bin Index'].isin(range(0,2))

        # Sample money laundering (ML) transactions.
        transactions_ml = transactions_ml[transactions_ml['ML Candicate'] == True].sample(n=int(n*r), replace=False, random_state=42)

        # Sample a smurfed transaction size for each money laundering transactions.
        bin_map_inv = {v: k for k, v in bin_map.items()} # Inverse bin_map.
        b_l = bin_map_inv[-1].left # The the minimum size of any smurfed transaction.
        transactions_ml['Smurf Size'] = np.random.uniform(low=b_l,high=0,size=len(transactions_ml))

        # Calculate the number of smurf transactions associted with each money laundering transaction.
        # Utilizes the log quotient rule.
        transactions_ml['Number of Smurfs'] = np.exp(transactions_ml['Size']-transactions_ml['Smurf Size'])
        transactions_ml['Number of Smurfs'] = np.floor(transactions_ml['Number of Smurfs']).astype(int)

        # Make a list with the smurfed transactions.
        smurf_trans = []
        for index, row in transactions_ml.iterrows():
            for number in range(row['Number of Smurfs']):
                smurf_trans.append(row['Smurf Size'])

        # Convert to pandas dataframe.
        smurf_trans = pd.DataFrame(data= {'Size': smurf_trans})

        # Remove money laundering transactions and add the new smurf transactions.
        transactions = transactions[~transactions.index.isin(transactions_ml.index)]
        transactions = pd.concat([transactions, smurf_trans])

        print(f'Added {len(smurf_trans)} money laundering transactions.')
        print(f'Removed {int(n*r)} smurfed transactions.')

        ######### Run analysis. #########
        zeta = estimate_zeta(transactions, l=-1, u=2, plot=True, fig_name=(f'sim_'+param['Type']+f'_r={r}'))
        print(f'Estimated zeta={zeta*100}')
        
        if 'A' in param['Type']:
            ls, us, res = [-1], [1,2], []
        if 'B' in param['Type']:
            ls, us, res = [-1], [1,2,3], []

        for l in ls:
            for u in us:
                # Run analysis.
                zeta = estimate_zeta(transactions, l=l, u=u, plot=False)
                ci_lower = bootstrap(transactions, l=l, u=u, iterations=10000)
                res.append([u, l, zeta, ci_lower])
                print(f'Done with #{len(res)}/{len(ls)*len(us)} (l={l}, u={u}), estimated: {zeta*100:.2f} [{ci_lower*100:.2f}].')

        res = pd.DataFrame(res, columns = ['u','l','zeta', 'CI Lower']) 
        print('')